In [13]:
import os  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

In [14]:
# Read in csv data
attrition_df = pd.read_csv("./data/IBM_attrition_data.csv")
data = attrition_df.copy()
data

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [15]:
data.sample(20)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
826,38,No,Travel_Rarely,433,Human Resources,1,3,Human Resources,1,1152,...,4,80,1,7,2,4,7,6,5,0
178,46,No,Travel_Rarely,526,Sales,1,2,Marketing,1,244,...,3,80,3,24,2,3,24,13,15,7
273,37,No,Travel_Rarely,228,Sales,6,4,Medical,1,378,...,2,80,1,7,5,4,5,4,0,1
281,42,No,Travel_Rarely,635,Sales,1,1,Life Sciences,1,387,...,3,80,0,20,3,3,20,16,11,6
577,37,No,Travel_Rarely,571,Research & Development,10,1,Life Sciences,1,802,...,2,80,2,6,3,2,5,3,4,3
1275,51,No,Travel_Rarely,942,Research & Development,3,3,Technical Degree,1,1786,...,4,80,0,15,2,3,2,2,2,2
889,27,No,Travel_Rarely,1103,Research & Development,14,3,Life Sciences,1,1244,...,4,80,2,9,3,2,9,7,6,8
517,25,No,Travel_Rarely,810,Sales,8,3,Life Sciences,1,707,...,3,80,1,4,4,3,3,2,1,2
1299,40,No,Travel_Rarely,1194,Research & Development,1,3,Life Sciences,1,1822,...,4,80,1,12,3,3,5,3,0,3
891,44,No,Travel_Rarely,1117,Research & Development,2,1,Life Sciences,1,1246,...,4,80,1,10,5,3,10,5,7,7


In [16]:
# Checking data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [19]:
# Drop Over18 column since all employees are over 18
data = data.drop(["Over18", "EmployeeCount", "StandardHours", "EmployeeNumber"], axis=1)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EnvironmentSatisfaction   1470 non-null   int64 
 9   Gender                    1470 non-null   object
 10  HourlyRate                1470 non-null   int64 
 11  JobInvolvement            1470 non-null   int64 
 12  JobLevel                  1470 non-null   int64 
 13  JobRole                   1470 non-null   object
 14  JobSatisfaction         

In [22]:
# # # Category Distribution 
# data[["BusinessTravel", "Department", "Education", 
#       "EducationField", "EnvironmentSatisfaction", "Gender",
#       "JobInvolvement", "JobRole", "JobSatisfaction", 
#       "MaritalStatus", "OverTime", "PerformanceRating", 
#       "RelationshipSatisfaction", "WorkLifeBalance"]].value_counts()/len(data)

In [ ]:
# sns.pairplot(data[[[["BusinessTravel", "Department", "Education", 
#                     "EducationField", "EnvironmentSatisfaction", "Gender",
#                     "JobInvolvement", "JobRole", "JobSatisfaction", 
#                     "MaritalStatus", "OverTime", "PerformanceRating", 
#                     "RelationshipSatisfaction", "WorkLifeBalance"]], diag_kind='kde']])

In [25]:
# # Rename Attrition_Yes column to Attrition 
# attrition_data = attrition_df.rename(columns={"Attrition_Yes": "Attrition", "OverTime_Yes": "OverTime", "Gender_Male": "Gender"})
# attrition_data.info()

In [24]:
# Set train and test set
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

test_set.shape

(294, 31)

In [27]:
# Convert integer classes in categorical columns to their names 
def preprocess_cat_columns(train_set):
    train_set["Education"] = train_set["Education"].map({1:"Below College", 2:"College", 3:"Bachelor", 4:"Master",5:"Doctor"}) 
    train_set["EnvironmentSatisfaction"] = train_set["EnvironmentSatisfaction"].map({1:"Low", 2:"Medium", 3:"High", 4:"Very High"})
    train_set["JobInvolvement"] = train_set["JobInvolvement"].map({1:"Low", 2:"Medium", 3:"High", 4:"Very High"})
    train_set["JobSatisfaction"] = train_set["JobSatisfaction"].map({1:"Low", 2:"Medium", 3:"High", 4:"Very High"})
    train_set["PerformanceRating"] = train_set["PerformanceRating"].map({1:"Low", 2:"Medium", 3:"High", 4:"Very High"})
    train_set["RelationshipSatisfaction"] = train_set["RelationshipSatisfaction"].map({1:"Low", 2:"Medium", 3:"High", 4:"Very High"})
    train_set["WorkLifeBalance"] = train_set["WorkLifeBalance"].map({1:"Bad", 2:"Good", 3:"Better", 4:"Best"})
    return train_set


data_tr = preprocess_cat_columns(train_set)
data_tr.head()

C:\Users\jenny\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jenny\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\jenny\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set 

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1097,24,No,Travel_Rarely,350,Research & Development,21,College,Technical Degree,High,Male,...,High,Medium,3,2,3,Better,1,1,0,0
727,18,No,Non-Travel,287,Research & Development,5,College,Life Sciences,Medium,Male,...,High,Very High,0,0,2,Better,0,0,0,0
254,29,No,Travel_Rarely,1247,Sales,20,College,Marketing,Very High,Male,...,High,Very High,1,10,2,Better,3,2,0,2
1175,39,No,Travel_Rarely,492,Research & Development,12,Bachelor,Medical,Very High,Male,...,Very High,High,0,7,3,Better,5,4,1,0
1341,31,No,Travel_Rarely,311,Research & Development,20,Bachelor,Life Sciences,Medium,Male,...,High,Low,1,10,2,Better,10,8,0,2


In [28]:
# One hot encoding the categorical columns 
train_set = pd.get_dummies(train_set, prefix='', prefix_sep='')
train_set

,Age,DailyRate,DistanceFromHome,HourlyRate,JobLevel,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,...,High,Very High,High,Low,Medium,Very High,Bad,Best,Better,Good
1097,24,350,21,57,1,2296,10036,0,14,3,...,1,0,0,0,1,0,0,0,1,0
727,18,287,5,73,1,1051,13493,1,15,0,...,1,0,0,0,0,1,0,0,1,0
254,29,1247,20,45,2,6931,10732,2,14,1,...,1,0,0,0,0,1,0,0,1,0
1175,39,492,12,66,2,5295,7693,4,21,0,...,0,1,1,0,0,0,0,0,1,0
1341,31,311,20,89,2,4197,18624,1,11,1,...,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,35,750,28,46,2,3407,25348,1,17,2,...,1,0,0,0,0,1,0,0,0,1
1294,41,447,5,85,2,6870,15530,3,12,0,...,1,0,0,1,0,0,1,0,0,0
860,22,1256,3,48,1,2853,4223,0,11,1,...,1,0,0,0,1,0,0,0,1,0
1459,29,1378,13,46,2,4025,23679,4,13,1,...,1,0,0,1,0,0,0,0,1,0


In [29]:
data = train_set.copy()

In [13]:
# Analyze correlation of each variable with the target variable in the train set 
train_data = train_set.copy()
corr_matrix = train_data.corr()
corr_matrix["Attrition"].sort_values(ascending=False)

Attrition                            1.000000
OverTime                             0.280567
MaritalStatus_Single                 0.205061
JobRole_Sales Representative         0.170863
BusinessTravel_Travel_Frequently     0.112736
JobRole_Laboratory Technician        0.099943
Department_Sales                     0.084838
DistanceFromHome                     0.067886
EducationField_Technical Degree      0.064842
NumCompaniesWorked                   0.063013
EducationField_Marketing             0.045301
Gender                               0.036962
JobRole_Human Resources              0.035862
MonthlyRate                          0.019092
JobRole_Research Scientist           0.012676
JobRole_Sales Executive              0.012341
HourlyRate                           0.003221
PerformanceRating                   -0.005086
EducationField_Other                -0.005969
PercentSalaryHike                   -0.017458
RelationshipSatisfaction            -0.018682
Education                         

In [174]:
# Setting target and feature variables in the training data set
X = train_set.drop(["Attrition"], axis=1)
y = train_set["Attrition"].copy()
X

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime
1097,24,350,21,2,3,57,2,1,1,2296,...,1,0,0,0,0,0,0,0,0,0
727,18,287,5,2,2,73,3,1,4,1051,...,0,0,0,0,1,0,0,0,1,0
254,29,1247,20,2,4,45,3,2,4,6931,...,0,0,0,0,0,1,0,0,0,0
1175,39,492,12,3,4,66,3,2,2,5295,...,0,0,1,0,0,0,0,1,0,0
1341,31,311,20,3,2,89,3,2,3,4197,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,35,750,28,3,2,46,4,2,3,3407,...,1,0,0,0,0,0,0,1,0,0
1294,41,447,5,3,2,85,4,2,2,6870,...,0,0,0,0,0,0,0,0,1,0
860,22,1256,3,4,3,48,2,1,4,2853,...,0,0,0,0,1,0,0,1,0,1
1459,29,1378,13,2,4,46,2,2,2,4025,...,1,0,0,0,0,0,0,1,0,1
